# Cloud Workshop Azure Databricks
## 06. Classification
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 29/06/2020

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Creating a Classification Model

In this exercise, you will implement a classification model that uses features of a flight to predict whether or not it will be late.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [4]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:18:15.357544

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights that has already been cleaned up for modeling. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating whether or not the flight was late (a flight with an arrival delay of more than 25 minutes is considered *late*).

You will load this data into a dataframe and display it.

> Importer le fichier CSV dans DBFS

In [8]:
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [10]:
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

train_rows = train.count()
test_rows = test.count()

print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 446159 Testing Rows: 190924

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **Late** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to define the training dataset.*

In [12]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column

carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")

training = assembler.transform(numTrain).select(col("features"), col("Late").alias("label"))

training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 1|
[1.0,1.0,10.0,103...| 0|
[1.0,1.0,10.0,105...| 1|
[1.0,1.0,10.0,107...| 0|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,110...| 1|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,114...| 1|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,124...| 1|
+--------------------+-----+
only showing top 20 rows

### Train a Classification Model
Next, you need to train a classification model using the training data. To do this, create an instance of the classification algorithm you want to use and use its **fit** method to train a model based on the training dataframe. In this exercise, you will use a *Logistic Regression* classification algorithm - but you can use the same technique for any of the classification algorithms supported in the spark.ml API.

In [14]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)

print ("OK !")

OK !

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **Late** column to **trueLabel**.

In [16]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), col("Late").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,112...| 1|
[1.0,1.0,10.0,114...| 0|
[1.0,1.0,10.0,114...| 1|
[1.0,1.0,10.0,123...| 1|
[1.0,1.0,10.0,123...| 0|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,132...| 0|
[1.0,1.0,10.0,132...| 0|
[1.0,1.0,10.0,134...| 1|
[1.0,1.0,10.0,139...| 1|
[1.0,1.0,10.0,139...| 1|
[1.0,1.0,10.0,153...| 0|
[1.0,1.0,2.0,1072...| 0|
[1.0,1.0,2.0,1072...| 1|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict delay status for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted status to the actual status.

In [18]:
prediction = model.transform(testing)
predicted = prediction.select("features", "probability", col("prediction").cast("Int"), "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------+----------------------------------------+----------+---------+
features |probability |prediction|trueLabel|
+------------------------------------+----------------------------------------+----------+---------+
[1.0,1.0,10.0,11193.0,10821.0,62.0] |[0.3003064214890074,0.6996935785109926] |1 |1 |
[1.0,1.0,10.0,11193.0,11298.0,55.0] |[0.3291452405832434,0.6708547594167567] |1 |1 |
[1.0,1.0,10.0,11193.0,13342.0,55.0] |[0.33462372004485635,0.6653762799551436]|1 |1 |
[1.0,1.0,10.0,11278.0,12478.0,58.0] |[0.32048721976826283,0.6795127802317372]|1 |1 |
[1.0,1.0,10.0,11433.0,10423.0,-3.0] |[0.5844994415455392,0.41550055845446077]|0 |0 |
[1.0,1.0,10.0,11433.0,12266.0,1.0] |[0.572092981602234,0.42790701839776596] |0 |1 |
[1.0,1.0,10.0,12339.0,11433.0,-6.0] |[0.6035559575817968,0.39644404241820325]|0 |1 |
[1.0,1.0,10.0,12339.0,14492.0,-12.0]|[0.6380733486848148,0.3619266513151852] |0 |0 |
[1.0,1.0,10.0,12478.0,10821.0,29.0] |[0.44389777937023844,0.5561022206297616]|1 |0 |
[1.0,1.0,10.0,12478.0,10821.0,44.0] |[0.3776145225889311,0.622385477411069] |1 |1 |
[1.0,1.0,10.0,12478.0,11278.0,89.0] |[0.21129361764701135,0.7887063823529887]|1 |1 |
[1.0,1.0,10.0,12478.0,11433.0,-1.0] |[0.5819286620741871,0.41807133792581286]|0 |0 |
[1.0,1.0,10.0,13232.0,11433.0,-4.0] |[0.5976091159909612,0.4023908840090387] |0 |0 |
[1.0,1.0,10.0,13244.0,10423.0,-6.0] |[0.6034927403574374,0.39650725964256267]|0 |0 |
[1.0,1.0,10.0,13487.0,14683.0,65.0] |[0.3049693757264856,0.6950306242735144] |1 |1 |
[1.0,1.0,10.0,13930.0,11193.0,43.0] |[0.3875229152058711,0.6124770847941289] |1 |1 |
[1.0,1.0,10.0,13931.0,12478.0,62.0] |[0.31223898606117473,0.6877610139388253]|1 |1 |
[1.0,1.0,10.0,15304.0,14492.0,-9.0] |[0.6344323217504163,0.36556767824958375]|0 |0 |
[1.0,1.0,2.0,10721.0,13303.0,-8.0] |[0.6165135888223218,0.3834864111776782] |0 |0 |
[1.0,1.0,2.0,10721.0,13303.0,18.0] |[0.4998186092064043,0.5001813907935957] |1 |1 |
[1.0,1.0,2.0,10721.0,13303.0,41.0] |[0.3961909487760249,0.6038090512239751] |1 |1 |
[1.0,1.0,2.0,10821.0,11298.0,52.0] |[0.3440151991006986,0.6559848008993013] |1 |1 |
[1.0,1.0,2.0,11042.0,11298.0,-1.0] |[0.5809701124412552,0.41902988755874476]|0 |0 |
[1.0,1.0,2.0,11298.0,10423.0,78.0] |[0.2450415822601266,0.7549584177398734] |1 |1 |
[1.0,1.0,2.0,11298.0,10693.0,0.0] |[0.5755505595026935,0.4244494404973065] |0 |0 |
[1.0,1.0,2.0,11298.0,10821.0,-1.0] |[0.5803888722325451,0.41961112776745496]|0 |0 |
[1.0,1.0,2.0,11298.0,11042.0,10.0] |[0.5314254092976397,0.4685745907023603] |0 |0 |
[1.0,1.0,2.0,11298.0,11057.0,-2.0] |[0.5855286366267558,0.4144713633732441] |0 |0 |
[1.0,1.0,2.0,11298.0,12264.0,-4.0] |[0.5978893163854893,0.40211068361451074]|0 |0 |
[1.0,1.0,2.0,11298.0,12889.0,-3.0] |[0.5953060700890614,0.40469392991093867]|0 |0 |
[1.0,1.0,2.0,11298.0,12889.0,34.0] |[0.4278249326180078,0.5721750673819923] |1 |1 |
[1.0,1.0,2.0,11298.0,12892.0,75.0] |[0.2610488357116431,0.7389511642883568] |1 |1 |
[1.0,1.0,2.0,11298.0,12953.0,-2.0] |[0.5910794880561338,0.40892051194386625]|0 |0 |
[1.0,1.0,2.0,11298.0,13495.0,-1.0] |[0.5882396159231169,0.41176038407688315]|0 |0 |
[1.0,1.0,2.0,11298.0,13851.0,44.0] |[0.3865143848434915,0.6134856151565085] |1 |1 |
[1.0,1.0,2.0,11298.0,13891.0,65.0] |[0.30036438047827385,0.6996356195217263]|1 |1 |
[1.0,1.0,2.0,11298.0,13930.0,27.0] |[0.46253838167738315,0.5374616183226169]|1 |0 |
[1.0,1.0,2.0,11298.0,14679.0,0.0] |[0.5872761044648036,0.41272389553519634]|0 |0 |
[1.0,1.0,2.0,11298.0,14683.0,22.0] |[0.48760677976647093,0.5123932202335291]|1 |0 |
[1.0,1.0,2.0,11298.0,14831.0,6.0] |[0.560901713680782,0.439098286319218] |0 |0 |
[1.0,1.0,2.0,11298.0,15304.0,49.0] |[0.3691495998894215,0.6308504001105785] |1 |1 |
[1.0,1.0,2.0,12339.0,13930.0,65.0] |[0.3033591858679514,0.6966408141320486] |1 |1 |
[1.0,1.0,2.0,12478.0,13303.0,-1.0] |[0.5914459242328569,0.408554075767143] |0 |0 |
[1.0,1.0,2.0,12892.0,11298.0,50.0] |[0.35857809969369275,0.6414219003063073]|1 |1 |
[1.0,1.0,2.0,12892.0,13303.0,132.0] |[0.1133505

In [19]:
display(predicted)

features,probability,prediction,trueLabel
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 10821.0, 62.0))","List(1, 2, List(), List(0.3003064214890074, 0.6996935785109926))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","List(1, 2, List(), List(0.3291452405832434, 0.6708547594167567))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 13342.0, 55.0))","List(1, 2, List(), List(0.33462372004485635, 0.6653762799551436))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11278.0, 12478.0, 58.0))","List(1, 2, List(), List(0.32048721976826283, 0.6795127802317372))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 10423.0, -3.0))","List(1, 2, List(), List(0.5844994415455392, 0.41550055845446077))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 12266.0, 1.0))","List(1, 2, List(), List(0.572092981602234, 0.42790701839776596))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 11433.0, -6.0))","List(1, 2, List(), List(0.6035559575817968, 0.39644404241820325))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 14492.0, -12.0))","List(1, 2, List(), List(0.6380733486848148, 0.3619266513151852))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 29.0))","List(1, 2, List(), List(0.44389777937023844, 0.5561022206297616))",1,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 44.0))","List(1, 2, List(), List(0.3776145225889311, 0.622385477411069))",1,1


In [20]:
display(predicted)

features,probability,prediction,trueLabel
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 10821.0, 62.0))","List(1, 2, List(), List(0.3003064214890074, 0.6996935785109926))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","List(1, 2, List(), List(0.3291452405832434, 0.6708547594167567))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 13342.0, 55.0))","List(1, 2, List(), List(0.33462372004485635, 0.6653762799551436))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11278.0, 12478.0, 58.0))","List(1, 2, List(), List(0.32048721976826283, 0.6795127802317372))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 10423.0, -3.0))","List(1, 2, List(), List(0.5844994415455392, 0.41550055845446077))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 12266.0, 1.0))","List(1, 2, List(), List(0.572092981602234, 0.42790701839776596))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 11433.0, -6.0))","List(1, 2, List(), List(0.6035559575817968, 0.39644404241820325))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 14492.0, -12.0))","List(1, 2, List(), List(0.6380733486848148, 0.3619266513151852))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 29.0))","List(1, 2, List(), List(0.44389777937023844, 0.5561022206297616))",1,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 44.0))","List(1, 2, List(), List(0.3776145225889311, 0.622385477411069))",1,1


In [21]:
display(predicted)

features,probability,prediction,trueLabel
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 10821.0, 62.0))","List(1, 2, List(), List(0.3003064214890074, 0.6996935785109926))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","List(1, 2, List(), List(0.3291452405832434, 0.6708547594167567))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11193.0, 13342.0, 55.0))","List(1, 2, List(), List(0.33462372004485635, 0.6653762799551436))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11278.0, 12478.0, 58.0))","List(1, 2, List(), List(0.32048721976826283, 0.6795127802317372))",1,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 10423.0, -3.0))","List(1, 2, List(), List(0.5844994415455392, 0.41550055845446077))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 11433.0, 12266.0, 1.0))","List(1, 2, List(), List(0.572092981602234, 0.42790701839776596))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 11433.0, -6.0))","List(1, 2, List(), List(0.6035559575817968, 0.39644404241820325))",0,1
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12339.0, 14492.0, -12.0))","List(1, 2, List(), List(0.6380733486848148, 0.3619266513151852))",0,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 29.0))","List(1, 2, List(), List(0.44389777937023844, 0.5561022206297616))",1,0
"List(1, 6, List(), List(1.0, 1.0, 10.0, 12478.0, 10821.0, 44.0))","List(1, 2, List(), List(0.3776145225889311, 0.622385477411069))",1,1
